In [ ]:
# 필요한 모듈 임포트
import requests
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from bs4 import *
import ssl
ssl._create_default_https_context = ssl._create_unverified_context
import warnings
import re
warnings.filterwarnings('ignore')

# yy, mm만 바꿔서 날짜조절. mm에서 09가 아니라 9인 거 확인
web_url = "https://www.weather.go.kr/w/obs-climate/land/past-obs/obs-by-day.do?stn=108&yy=2022&mm=9&obs=1"

# 크롤링으로 데이터 가져오기
req = requests.get(web_url, headers={"User-agent":"Mozilla/5.0"})
soup = BeautifulSoup(req.text, 'html.parser')
trList = list(map(lambda x:x.get_text(), soup.find_all('tr')))

# 데이터 예쁘게 깎아서 담기
days, data = [], []
for row in trList[5::2]:
    data.extend([float(*re.findall(r'\d+.\d+', item)) for item in row.split('\n') if len(item) and item != '\xa0'])
for row in trList[4::2]:
    days.extend([item for item in row.split('\n') if len(item) and item != '\xa0'])

# 판다스 데이터프레임에 담고 출력. 이건 주피터노트북 환경이라 df 했는데, .py에선 print(df) 해줘야함
df = pd.DataFrame(index=days, columns=['평균기온', '최고기온', '최저기온', '평균운량', '일강수량'])
df['평균기온'] = data[0::5]
df['최고기온'] = data[1::5]
df['최저기온'] = data[2::5]
df['평균운량'] = data[3::5]
df['일강수량'] = data[4::5]
df